In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

test = pd.read_csv("data/idsc_dataset.csv", delimiter=";")

In [2]:
test["data"] = pd.to_datetime(test["dt_dep"])

/tmp/ipykernel_3252060/180993028.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test["data"] = pd.to_datetime(test["dt_dep"])


In [4]:
len(test["path"].unique())

969

In [10]:
import requests

def download_images(df):
    unlock = False
    for row in df["path"].unique():

        # Faz o download da imagem usando a biblioteca requests
        try:
            data = row.split('/')[-1]
            path = row
            
            # Extrai o nome do arquivo da URL
            nome_arquivo = path.split('/')[-1]
            print(nome_arquivo)
    
            # Define o caminho onde a imagem será salva
            pasta_destino = f'data/img/test/'
        
            response = requests.get(path)
        
            # Verifica se a resposta foi bem sucedida (código 200)
            if response.status_code == 200:
                with open(f'{pasta_destino}{data}.jpg', 'wb') as file:
                    file.write(response.content)
                    print(f"Imagem {nome_arquivo} salva em {pasta_destino}")
            else:
                print(f"Erro ao baixar a imagem {nome_arquivo}")
        except:
            pass


In [11]:
download_images(test)

S11635384_202308211100.jpg
Imagem S11635384_202308211100.jpg salva em data/img/test/
S11635384_202309150000.jpg
Imagem S11635384_202309150000.jpg salva em data/img/test/
S11635384_202309030100.jpg
Imagem S11635384_202309030100.jpg salva em data/img/test/
S11635384_202309150300.jpg
Imagem S11635384_202309150300.jpg salva em data/img/test/
S11635384_202308230000.jpg
Imagem S11635384_202308230000.jpg salva em data/img/test/
S11635384_202309130900.jpg
Imagem S11635384_202309130900.jpg salva em data/img/test/
S11635384_202308210900.jpg
Imagem S11635384_202308210900.jpg salva em data/img/test/
S11635384_202309111300.jpg
Imagem S11635384_202309111300.jpg salva em data/img/test/
S11635384_202309132000.jpg
Imagem S11635384_202309132000.jpg salva em data/img/test/
S11635384_202309012200.jpg
Imagem S11635384_202309012200.jpg salva em data/img/test/
S11635384_202309161300.jpg
Imagem S11635384_202309161300.jpg salva em data/img/test/
S11635384_202308302300.jpg
Imagem S11635384_202308302300.jpg salv

In [16]:
import os
import subprocess
import pandas as pd
import json
from multiprocessing import Pool, cpu_count

# Function to be mapped
def process_file(file):
    log = []
    data_list = []
    file_path = os.path.join('data/img/test/', file)
    print(file)
    try:
        output = subprocess.check_output(['python3', 'utils/decodeImage.py', file_path])
        output = json.loads(output)
        
        polyCount = 0
        for poly in output:
            verticeCount = 0
            for vertice in poly:
                x = vertice[0][0]
                y = vertice[0][1]
                data_list.append({'data': file, 'X': x, 'Y': y, 'poly': polyCount, 'vertice': verticeCount})
                verticeCount = verticeCount + 1
            polyCount = polyCount + 1
        log.append(f'{file}: sem erros\n')
    except subprocess.CalledProcessError as err:
        log.append(f'{file}: erro na execução - {err}\n')
    
    return data_list, log


# Use Pool to create subprocesses
with Pool(min(8, cpu_count())) as p:
    results = p.map(process_file, os.listdir('data/img/test/'))

# Combine results
all_data = []
all_logs = []
for data, log in results:
    all_data.extend(data)
    all_logs.extend(log)

# Creates dataframe from the data
dataset = pd.DataFrame(all_data)

# Cria o arquivo de log
with open('log/log2.txt', 'w') as log_file:
    log_file.writelines(all_logs)

# Export dataframe to csv
dataset.to_csv('data/testImageData.csv', index=False)


S11635384_202309110900.jpg.jpgS11635384_202308242300.jpg.jpgS11635384_202308151600.jpg.jpgS11635384_202309140500.jpg.jpgS11635384_202308261900.jpg.jpg




S11635384_202309022300.jpg.jpgS11635384_202309110500.jpg.jpg
S11635384_202309181200.jpg.jpg

S11635384_202309182300.jpg.jpg
S11635384_202308291600.jpg.jpg
S11635384_202308310500.jpg.jpg
S11635384_202309101100.jpg.jpg
S11635384_202309031000.jpg.jpg
S11635384_202308191800.jpg.jpg
S11635384_202308170700.jpg.jpg
S11635384_202308201700.jpg.jpg
S11635384_202309051200.jpg.jpg
S11635384_202308240100.jpg.jpg
S11635384_202309091700.jpg.jpg
S11635384_202308150300.jpg.jpg
S11635384_202309210800.jpg.jpg
S11635384_202309202300.jpg.jpg
S11635384_202309171600.jpg.jpg
S11635384_202309061000.jpg.jpg
S11635384_202309231400.jpg.jpg
S11635384_202308181100.jpg.jpg
S11635384_202308211500.jpg.jpg
S11635384_202309012000.jpg.jpg
S11635384_202308250100.jpg.jpg
S11635384_202309142200.jpg.jpg
S11635384_202308260900.jpg.jpg
S11635384_202309010500.jpg.jpg
S1163538

In [3]:
import pandas as pd
dataset = pd.read_csv('data/testImageData.csv')

In [8]:
import pandas as pd
from shapely.geometry import Polygon

#dataset['X'] = dataset["X"].apply(lambda x: x+940)
#dataset['Y'] = dataset["Y"].apply(lambda x: x+740)
polygons = dataset.groupby(['data', 'poly'])[['X', 'Y']].apply(lambda x: Polygon(x.values))

# Calculate the centroids
df_centroids = polygons.apply(lambda polygon: polygon.centroid)
#
## Estimate the area
df_area = polygons.apply(lambda polygon: polygon.area)
#
## Create a DataFrame with the results
results = pd.DataFrame({
    'polygon': polygons.index,
    'centroid_x': [p.x for p in df_centroids],  
    'centroid_y': [p.y for p in df_centroids], 
    'area_estimate': df_area.values
})


In [9]:
results

,polygon,centroid_x,centroid_y,area_estimate
0,"(S11635384_202308150100.jpg.jpg, 0)",351.842365,1304.818924,148947.0
1,"(S11635384_202308150100.jpg.jpg, 1)",96.973030,1105.140735,52837.0
2,"(S11635384_202308150100.jpg.jpg, 2)",996.431988,1103.493383,31167.5
3,"(S11635384_202308150100.jpg.jpg, 3)",767.379381,213.143635,10684.5
4,"(S11635384_202308150100.jpg.jpg, 4)",692.609052,135.450677,11820.0
...,...,...,...,...
409030,"(S11635384_202309250000.jpg.jpg, 464)",1.500000,234.000000,12.0
409031,"(S11635384_202309250000.jpg.jpg, 465)",821.500000,1.000000,10.0
409032,"(S11635384_202309250000.jpg.jpg, 466)",949.000000,1450.000000,8.0
409033,"(S11635384_202309250000.jpg.jpg, 467)",385.000000,1450.000000,8.0


In [10]:
results["poly"] = results["polygon"].apply(lambda x: str(x).split(",")[1].split(")")[0].replace(" ","").replace("(","").replace("'",""))
results["data"] = pd.to_datetime(results["polygon"].apply(lambda x: str(x).split(",")[0].split("_")[1].split(".")[0]))

In [11]:
results["centroid_x"] = results["centroid_x"].apply(lambda x: x + 740)
results["centroid_y"] = results["centroid_y"].apply(lambda x: x + 940)

In [12]:


import numpy as np
from scipy import interpolate

def create_transform(pixel_coords, latlon_coords):

    pixel_x, pixel_y = zip(*pixel_coords)
    lat, lon = zip(*latlon_coords)

    lat_interp = interpolate.interp1d(pixel_y, lat, fill_value="extrapolate")
    lon_interp = interpolate.interp1d(pixel_x, lon, fill_value="extrapolate")

    return lat_interp, lon_interp

def convert_pixel_to_latlong(x, y):
    lat_lon_points = [(7.455477, -77.829943), (-51.848798, -76.538633), (1.355664, -28.966528), (8.000000, -29.500000)]
    x_y_points = [(940, 740), (940, 2192), (2092, 2192), (2092, 740)]
    lat_interp, lon_interp = create_transform(x_y_points, lat_lon_points)
    return lat_interp(y), lon_interp(x)


# Para converter um ponto de pixel para lat/lon:
lat, lon = convert_pixel_to_latlong(1500, 1500) # -3.152260, -40.788782
print(lat, lon)

results["lat"] = results.apply(lambda x: convert_pixel_to_latlong(x["centroid_x"], x["centroid_y"])[1], axis=1)
results["lon"] = results.apply(lambda x: convert_pixel_to_latlong(x["centroid_x"], x["centroid_y"])[0], axis=1)
#dataset["lat"] = dataset.apply(lambda x: convert_pixel_to_latlong(x["X"], x["Y"])[1], axis=1)
#dataset["lon"] = dataset.apply(lambda x: convert_pixel_to_latlong(x["X"], x["Y"])[0], axis=1)

-23.325817134986227 -53.413304180555556


/home/bruno/.local/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/bruno/.local/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:704: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
/home/bruno/.local/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:701: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/bruno/.local/lib/python3.10/site-packages/scipy/interpolate/_interpolate.py:704: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


In [13]:
results

,polygon,centroid_x,centroid_y,area_estimate,poly,data,lat,lon
0,"(S11635384_202308150100.jpg.jpg, 0)",1091.842365,2244.818924,148947.0,0,2023-08-15 01:00:00,-70.2682676285446,inf
1,"(S11635384_202308150100.jpg.jpg, 1)",836.973030,2045.140735,52837.0,1,2023-08-15 01:00:00,-inf,-45.79552630397732
2,"(S11635384_202308150100.jpg.jpg, 2)",1736.431988,2043.493383,31167.5,2,2023-08-15 01:00:00,-43.64979082996639,-45.72762544446794
3,"(S11635384_202308150100.jpg.jpg, 3)",1507.379381,1153.143635,10684.5,3,2023-08-15 01:00:00,-53.10857093032507,-9.029028889634912
4,"(S11635384_202308150100.jpg.jpg, 4)",1432.609052,1075.450677,11820.0,4,2023-08-15 01:00:00,-56.19622886174746,-5.826666526102398
...,...,...,...,...,...,...,...,...
409030,"(S11635384_202309250000.jpg.jpg, 464)",741.500000,1174.000000,12.0,464,2023-09-25 00:00:00,-inf,-9.888690311294766
409031,"(S11635384_202309250000.jpg.jpg, 465)",1561.500000,941.000000,10.0,465,2023-09-25 00:00:00,-50.87364753342014,-0.2848542685950406
409032,"(S11635384_202309250000.jpg.jpg, 466)",1689.000000,2390.000000,8.0,466,2023-09-25 00:00:00,-45.60850570399306,inf
409033,"(S11635384_202309250000.jpg.jpg, 467)",1125.000000,2390.000000,8.0,467,2023-09-25 00:00:00,-68.89901544357639,inf


In [27]:
results.to_csv("data/test_storm_areas", index=False)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('my-cool-app') \
    .getOrCreate()
spark.conf.set('spark.sql.crossJoin.enabled', 'true') 




In [2]:

# Assuming 'df_areas' is your first dataframe with centroid and 'df_flight' is your second dataframe
df_areas = spark.read.csv('./data/test_storm_areas', inferSchema=True, header=True)
# Register your dataframes as temp views to use them in SQL
df_areas.createOrReplaceTempView('df_areas')

In [3]:
df_flight = spark.read.csv('data/test_cat62.csv', inferSchema=True, header=True)

df_flight.createOrReplaceTempView('df_flight')


In [1]:
import numpy as np
import pandas as pd
df_flight = pd.read_csv('data/test_cat62.csv')
df_areas = pd.read_csv('./data/test_storm_areas')
# Register your dataframes as temp views to use them in SQL
# Assuming df_flight and df_areas are your dataframes
from math import sin, cos, sqrt, asin, radians

def haversine(row):
     # The radius of the earth in km
    R = 6371 
    lat1 = radians(row['f_lat'])
    lat2 = radians(row['a_lat'])
    dlat = lat2 - lat1
    dlon = radians(row['a_lon'] - row['f_lon'])
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

# Assuming df_flight and df_areas are your dataframes
df_flight['rounded_event_time'] = df_flight['data'].apply(lambda x: str(x)[:13])
df_areas['data'] = df_areas['data'].apply(lambda x: str(x)[:13])


In [4]:
df_flight = df_flight[['rounded_event_time', 'lat', 'lon', 'flightid']]
df_areas = df_areas[['data', 'lat', 'lon', 'area_estimate']]

In [5]:

# Join the dataframes on rounded_event_time
merged_df = pd.merge(df_flight, df_areas, left_on='rounded_event_time', right_on='data', 
                     suffixes=('_f', '_a'), how='left')

In [4]:


# Create 'flight_through_area' column
merged_df['flight_through_area'] = np.where(haversine(merged_df) <= merged_df['area_estimate'] / 2, 1, 0)

#Group by flightid
grouped_df = merged_df.groupby(['flightid'])

#Get the first of polygon and sum of flight_through_area
result_df = grouped_df.agg({'polygon' : lambda x: str(x.values[0])[2:13], 
                            'flight_through_area' : 'sum'}).reset_index()




In [4]:
from pyspark.sql.functions import *
#df_flight = df_flight.withColumn("dt_radar", to_timestamp(df_flight["dt_radar"] / 1000))

AnalysisException: [DATATYPE_MISMATCH.BINARY_OP_DIFF_TYPES] Cannot resolve "(dt_radar / 1000)" due to data type mismatch: the left and right operands of the binary operator have incompatible types ("TIMESTAMP" and "INT").;
'Project [flightid#50, origem#51, destino#52, hora_ref#53, path#54, dt_dep#55, linha#56, lat#57, lon#58, flightlevel#59, speed#60, to_timestamp((dt_radar#61 / 1000), None, TimestampType, Some(America/Sao_Paulo), false) AS dt_radar#76, data#62]
+- Relation [flightid#50,origem#51,destino#52,hora_ref#53,path#54,dt_dep#55,linha#56,lat#57,lon#58,flightlevel#59,speed#60,dt_radar#61,data#62] csv


In [4]:

df_flight.createOrReplaceTempView('df_flight')

In [6]:
df_areas.show()

+-------------------+-------------------+-------------------+-------------+
|               data|                lat|                lon|area_estimate|
+-------------------+-------------------+-------------------+-------------+
|2023-08-15 01:00:00| -62.00922162160016| -45.78226186865345|     148947.0|
|2023-08-15 01:00:00| -72.53410940813013| -37.55189021582304|      52837.0|
|2023-08-15 01:00:00| -35.39074482302195|-37.483989356313664|      31167.5|
|2023-08-15 01:00:00| -44.84952492338063|-0.7853928014806364|      10684.5|
|2023-08-15 01:00:00| -47.93718285480301|  2.416969562051876|      11820.0|
|2023-08-15 01:00:00| -44.85776585732416| -51.06711036416402|       2851.0|
|2023-08-15 01:00:00|-60.975637760786526|-39.949590528724805|      17714.5|
|2023-08-15 01:00:00| -57.75871085196734| -32.59282422155726|       4980.0|
|2023-08-15 01:00:00| -42.04390638553994| -48.95924154875727|       1895.5|
|2023-08-15 01:00:00| -65.82364615749901|-39.318429095739496|      16173.5|
|2023-08-15 

In [7]:
df_flight.show()

+-------------------+-------------------+-------------------+--------------------+
|               data|                lat|                lon|            flightid|
+-------------------+-------------------+-------------------+--------------------+
|2023-08-21 11:00:00|-15.877232349332122| -47.90218536386029|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-16.076418514121805|  -47.8263104137048|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-16.461348423038604| -47.71525916662432|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-16.816224547009057|-47.611974024411744|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-17.459661084107587| -47.42379104425216|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-17.985411138976307| -47.33796428129105|4f0356600f61e3fcb...|
|2023-08-21 11:00:00|-18.502359507869475| -47.25515311552685|4f0356600f61e3fcb...|
|2023-08-21 11:00:00| -18.90474254201476| -47.19155572272939|4f0356600f61e3fcb...|
|2023-08-21 11:00:00| -19.42024302937089| -47.10585531988042|4f0356600f61e3fcb...|
|202

In [4]:
df_flight = df_flight.select('data', 'lat', 'lon', 'flightid')
df_areas = df_areas.select('data', 'lat', 'lon', 'area_estimate')


In [5]:
# Now run the SQL Query to join these two dataframes based on your conditions
query = """
    SELECT 
        f.flightid, first(SUBSTRING(CAST(a.polygon AS STRING), 3, 13)), 
    sum(CASE
            WHEN (2 * 6371 * ASIN(SQRT(POW(SIN((f.lat - a.lat) * .0174532925 / 2), 2) 
                                   + COS(f.lat * .0174532925) * COS(a.lat * .0174532925) *
                                   POW(SIN((f.lon - a.lon) * .0174532925 / 2), 2))))
                <= (a.area_estimate / 2) THEN 1
            ELSE 0
        END) as flight_through_area
    FROM (
        SELECT *,data,
        SUBSTRING(CAST(data AS STRING), 1, 13) AS rounded_event_time
        FROM df_flight
    ) f
    LEFT JOIN df_areas a
    ON f.data = a.data
    GROUP BY f.flightid
"""
result_df = spark.sql(query)

result_df.show()


+--------------------+------------------------------------------------+-------------------+
|            flightid|first(substring(CAST(polygon AS STRING), 3, 13))|flight_through_area|
+--------------------+------------------------------------------------+-------------------+
|00054c16230f8fb7c...|                                   S11635384_202|                552|
|0005c08c2b064c695...|                                   S11635384_202|                140|
|00062261eae0cd9c9...|                                   S11635384_202|                259|
|000625db56237c204...|                                   S11635384_202|                273|
|0007f2ee7329b979f...|                                   S11635384_202|                473|
|0008fd53ab12318d0...|                                   S11635384_202|                118|
|000a1ff7f66b8af8a...|                                   S11635384_202|                120|
|000cc25ac27a625ae...|                                   S11635384_202|         

In [8]:
result_df.repartition(1).write.format('csv').option('header',True).save('test_flight_area.csv')